# Generic Subprepare

In [ ]:
import cirq
import numpy as np
import cirq_qubitization.testing as cq_testing
from cirq_qubitization.jupyter_tools import display_gate_and_compilation
from typing import *

## Demo LCU Coefficients

We get some LCU coefficients for a 1D ising model to demonstrate subprepare.

$$
H = -J\sum_{k=0}^{L-1}\sigma_{k}^{Z}\sigma_{(k+1)\%L}^{Z} + -\Gamma \sum_{k=0}^{L-1}\sigma_{k}^{X}
$$

In [ ]:
def get_1d_ising_hamiltonian(
    qubits: Sequence[cirq.Qid], j_zz_strength: float = 1.0, gamma_x_strength: float = -1
) -> cirq.PauliSum:
    """A one dimensional ising model with periodic boundaries.

    $$
    H = -J\sum_{k=0}^{L-1}\sigma_{k}^{Z}\sigma_{(k+1)\%L}^{Z} + -\Gamma \sum_{k=0}^{L-1}\sigma_{k}^{X}
    $$

    Args:
        qubits: One qubit for each spin site.
        j_zz_strength: The two-body ZZ potential strength, $J$.
        gamma_x_strength: The one-body X potential strength, $\Gamma$.

    Returns:
        cirq.PauliSum representing the Hamiltonian
    """
    n_sites = len(qubits)
    terms = [
        cirq.PauliString(
            {qubits[k]: cirq.Z, qubits[(k + 1) % n_sites]: cirq.Z}, coefficient=j_zz_strength
        )
        for k in range(n_sites)
    ]
    terms.extend([cirq.PauliString({q: cirq.X}, coefficient=gamma_x_strength) for q in qubits])
    return cirq.PauliSum.from_pauli_strings(terms)

In [ ]:
spins = cirq.LineQubit.range(3)
ham = get_1d_ising_hamiltonian(spins, np.pi/3, np.pi/7)
coeffs = np.array([term.coefficient.real for term in ham])
coeffs

In [ ]:
lcu_coeffs = coeffs / np.sum(coeffs)
lcu_coeffs

## GenericSubPrepare

This gate implements generic sub-prepare defined in Fig 11 of https://arxiv.org/abs/1805.03662.

This corresponds to the following operations:
 - UNIFORM_L on first selection register
 - H^{mu} on mu-sigma-register
 - QROM-alt-keep selection is on first selection alt-keep are on next mu and logL registers
 - LessThanEqualGate
 - Coherent swap

Total space will be (2 * log(L) + 2 mu + 1) work qubits + log(L) ancillas for QROM.
The 1 ancilla in work qubits is for the `LessThanEqualGate` followed by coherent swap.

In [ ]:
from cirq_qubitization.generic_subprepare import GenericSubPrepare


g = cq_testing.GateSystem(
    GenericSubPrepare(lcu_coeffs, probability_epsilon=1e-2)
)

display_gate_and_compilation(g, vertical=True)